In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from joblib import dump

# 데이터 불러오기

In [2]:
train_x = pd.read_csv('./data/train_x_df.csv')
test_x = pd.read_csv('./data/test_x_df.csv')
train_y = pd.read_csv('./data/train_y_df.csv')

# 데이터 전처리
- 과거 1260분 데이터로 미래 1분을 예측하도록 하여 샘플당 120개의 행을 가지도록 전처리하여 학습 시키려 했으나 메모리 문제로 1320분 데이터로 2분을 예측하도록 했습니다.
- 과거 1320분 데이터로 미래 2분을 예측하도록 각 샘플의 x, y 데이터를 변환합니다. 
- ex) 0 ~ 1319분으로 0 ~ 1분 예측, 1 ~ 1320분으로 2 ~ 3분 예측, ...
- x: (1, 1380) -> (60, 1320)
- y: (1, 120) -> (60, 2)

In [3]:
# train 데이터셋을 변환하는 함수
def trans_train_dataset(x_df, y_df):
    # 샘플의 개수
    sample_cnt = train_x['sample_id'].nunique()
    # 행의 개수
    rows = 60
    # 열의 개수
    columns = 1320
    # 2분 예측
    pred_minutes = 2

    x = pd.DataFrame(np.zeros([rows * sample_cnt, columns]))
    y = pd.DataFrame(np.zeros([rows * sample_cnt, pred_minutes])) 
    
    # 샘플 개수 만큼 실행
    for sample_id in tqdm(range(sample_cnt)):
        tmp = x_df.query(f'sample_id == {sample_id}')
        # 행의 개수 만큼 실행
        for i in range(rows):  
            x.iloc[sample_id*rows + i] = tmp.iloc[i:i+columns]['open'].values
        y.iloc[sample_id*rows:sample_id*rows+rows, :] = np.array(y_df.query(f'sample_id == {sample_id}')['open']).reshape(rows, -1)
    
    return x, y

In [4]:
x_train, y_train = trans_train_dataset(train_x, train_y)

100%|██████████| 7661/7661 [14:17<00:00,  8.93it/s]


In [5]:
x_train.shape, y_train.shape

((459660, 1320), (459660, 2))

In [6]:
test_x['sample_id'].nunique()

535

In [7]:
# test 데이터셋을 변환하는 함수
def trans_test_dataset(x_df):
    # 샘플의 개수
    sample_cnt = test_x['sample_id'].nunique()
    # 행의 개수
    rows = 60
    # 열의 개수
    columns = 1320

    x = pd.DataFrame(np.zeros([rows * sample_cnt, columns]))
    
    # 샘플 개수 만큼 실행
    # test 데이터의 sample_id는 7661부터 시작
    for sample_id in tqdm(range(sample_cnt)):
        tmp = x_df.query(f'sample_id == {sample_id+7661}')
        # 행의 개수 만큼 실행
        for i in range(rows):  
            x.iloc[sample_id*rows + i] = tmp.iloc[i:i+columns]['open'].values
    
    return x

In [8]:
x_test = trans_test_dataset(test_x)

100%|██████████| 535/535 [00:16<00:00, 33.24it/s]


In [9]:
x_test.shape

(32100, 1320)

# 학습 및 예측

In [10]:
model = BaggingRegressor(LinearRegression())
model.fit(x_train, y_train)
pred = model.predict(x_test)

In [14]:
answer = pred.reshape(535, 120)

# 제출용 데이터프레임 생성

In [15]:
submission = pd.DataFrame()
submission['sample_id'] = range(train_x['sample_id'].nunique(), train_x['sample_id'].nunique()+test_x['sample_id'].nunique())
submission['buy_quantity'] = 1
submission['sell_time'] = np.argmax(answer, axis=1)
submission

,sample_id,buy_quantity,sell_time
0,7661,1,0
1,7662,1,113
2,7663,1,54
3,7664,1,115
4,7665,1,85
...,...,...,...
530,8191,1,4
531,8192,1,10
532,8193,1,61
533,8194,1,49


In [16]:
# submission.to_csv('submission_linear.csv', index=False)